## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from equilibrator_api import Q_, R, default_T
import model_balancing

In [2]:
# Network topology

S = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)
A_act = np.matrix(
    [
        [0, 1, 2],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=float
)
A_inh = np.matrix(
    [
        [0, 0, 0],
        [0, 0, 0],
        [2, 0, 0]
    ], dtype=float
)

In [3]:
# Priors for Kinetic and Thermodynamic parameters

Keq_gmean = np.array([57, 0.13, 3200])  # geometric mean (assuming a standard concentration of 1M)
Keq_ln_cov = np.array(
    [
        [4, 0, 0],
        [0, 4, 0],
        [0, 0, 4],
    ]
)  # log-scale covariance

kcatf_gmean = Q_([20.0, 8.0, 0.1], "1/s")  # geometric mean
kcatf_ln_cov = np.array(
    [
        [0.48, 0, 0],
        [0, 5.3, 0],
        [0, 0, 5.3],
    ]
) # log-scale covariance

kcatr_gmean = Q_([10.0, 10.0, 10.0], "1/s")  # geometric mean
kcatr_ln_cov = np.array(
    [
        [21.2, 0, 0],
        [0, 21.2, 0],
        [0, 0, 21.2],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
Km_gmean = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
Km_ln_cov = np.array(
    [
        [5.3, 0, 0, 0, 0, 0],
        [0, 5.3, 0, 0, 0, 0],
        [0, 0, 5.3, 0, 0, 0],
        [0, 0, 0, 5.3, 0, 0],
        [0, 0, 0, 0, 5.3, 0],
        [0, 0, 0, 0, 0, 5.3],
    ]
) # log-scale covariance

Ka_gmean = Q_([1, 1], "M")
Ka_ln_cov = np.array(
    [
        [21.2, 0],
        [0, 21.2],
    ]
)

Ki_gmean = Q_([1], "M")
Ki_ln_cov = np.array(
    [
        [21.2],
    ]
)

In [4]:
# def to_dense_matrix(S, mu: np.array) -> np.array:
#     Nc, Nr = S.shape
    
#     if mu.size == 0:
#         return np.zeros((Nc, Nr))

#     assert mu.ndim == 1
#     K_mat = []
#     k = 0
#     for i in range(Nc):
#         row = []
#         for j in range(Nr):
#             if S[i, j] != 0:
#                 row.append(mu[k])
#                 k += 1
#             else:
#                 row.append(0)
#         K_mat.append(np.hstack(row))
#     K_mat = np.vstack(K_mat)
#     return K_mat

# Km_gmean_full = to_dense_matrix(S, np.log(Km_gmean.m_as("M")))

# # use Haldane to find the k
# kcatr_gmean = np.exp(np.diag(S.T @ Km_gmean_full) + np.log(kcatf_gmean.m_as("1/s")) - np.log(Keq_gmean)) * Q_("1/s")
# kcatr_ln_cov = np.array(
#     [
#         [21.2, 0, 0],
#         [0, 21.2, 0],
#         [0, 0, 21.2],
#     ]
# ) # log-scale covariance


In [5]:
# condition dependent data (columns represent conditions)

fluxes = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

conc_enz_gmean = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

conc_enz_gstd = np.array(
    [
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
    ]
)  # geometric standard deviation

conc_met_gmean = Q_(
    [
        [1e-4, 1e-3, 1e-4, 1e-3],
        [1e-5, 1e-5, 1e-5, 1e-4],
        [1e-4, 1e-3, 1e-4, 1e-5],
    ], "M"
)  # geometric mean

conc_met_gstd = np.array(
    [
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
    ]
)  # geometric standard deviation

In [6]:
model_balancing.solve(
    S,
    fluxes,
    A_act,
    A_inh,
    Keq_gmean, Keq_ln_cov,
    conc_enz_gmean, conc_enz_gstd,
    conc_met_gmean, conc_met_gstd,
    kcatf_gmean, kcatf_ln_cov,
    kcatr_gmean, kcatr_ln_cov,
    Km_gmean, Km_ln_cov,
    Ka_gmean, Ka_ln_cov,
    Ki_gmean, Ki_ln_cov
)


Metabolite concentrations (M) =
 [[3.46812613e-04 1.99831443e-03 4.89547517e-04 1.83920507e-03]
 [5.07859337e-05 6.87125502e-05 4.05883348e-05 1.42008688e-04]
 [6.73102048e-05 1.59258278e-04 5.34581803e-05 9.99603775e-06]]

Enzyme concentrations (M) =
 [[0.00052168 0.00012085 0.00047948 0.00023144]
 [0.01312676 0.01038445 0.022323   0.00042749]
 [0.00540467 0.00078882 0.00233521 0.00218328]]

Driving forces (RT) =
 [[ 5.20565824  6.6546205   5.77449387  5.84569354]
 [ 0.67934535  0.12044238  0.68561882  3.61473723]
 [ 7.42654841  6.86764544  7.43282188 10.36194029]]

η(thr) =
 [[0.99 1.   1.   1.  ]
 [0.49 0.11 0.5  0.97]
 [1.   1.   1.   1.  ]]

η(kin) =
 [[0.16 0.5  0.21 0.44]
 [0.29 0.35 0.24 0.53]
 [0.31 0.37 0.26 0.55]]

η(reg) =
 [[1.   1.   1.   1.  ]
 [0.06 0.27 0.08 0.25]
 [0.26 0.74 0.36 0.72]]




All Z-scores
--------------------------------------------------
enzymes =  3.43
metabolites =  3.26
Keq =  2.43
kcat forward =  1.95
kcat reverse =  1.89
Km =  2.04
Ka =  4.31
Ki 

In [7]:
model_balancing.solve(
    S,
    fluxes,
    A_act,
    A_inh,
    Keq_gmean, Keq_ln_cov,
    conc_enz_gmean, conc_enz_gstd,
    conc_met_gmean, conc_met_gstd,
    kcatf_gmean, kcatf_ln_cov,
    kcatr_gmean, kcatr_ln_cov,
    Km_gmean, Km_ln_cov,
    Ka_gmean, Ka_ln_cov,
    Ki_gmean, Ki_ln_cov,
    rate_law = "1SP"
)


Metabolite concentrations (M) =
 [[3.82325275e-04 1.44004693e-03 5.76597166e-04 1.49537320e-03]
 [4.39153033e-05 7.83704559e-05 4.72583720e-05 1.50347324e-04]
 [6.31034662e-05 1.76744119e-04 5.49284228e-05 1.00011953e-05]]

Enzyme concentrations (M) =
 [[0.00055748 0.0001642  0.000499   0.0002988 ]
 [0.0131026  0.01113245 0.01365815 0.00043843]
 [0.00679707 0.00094561 0.00223699 0.00270158]]

Driving forces (RT) =
 [[ 5.41703961  6.16401468  5.75454476  5.55021369]
 [ 0.56791486  0.11717169  0.78002659  3.6406712 ]
 [ 7.33795925  6.88721609  7.55007099 10.41071559]]

η(thr) =
 [[1.   1.   1.   1.  ]
 [0.43 0.11 0.54 0.97]
 [1.   1.   1.   1.  ]]

η(kin) =
 [[0.14 0.37 0.2  0.34]
 [0.28 0.41 0.29 0.57]
 [0.3  0.43 0.31 0.59]]

η(reg) =
 [[1.   1.   1.   1.  ]
 [0.07 0.22 0.1  0.23]
 [0.25 0.63 0.36 0.64]]




All Z-scores
--------------------------------------------------
enzymes =  3.16
metabolites =  3.35
Keq =  2.4
kcat forward =  1.78
kcat reverse =  2.01
Km =  1.99
Ka =  4.24
Ki =

In [8]:
# All Z-scores
# --------------------------------------------------
# enzymes =  3.16
# metabolites =  3.35
# Keq =  2.4
# kcat forward =  1.78
# kcat reverse =  2.01
# Km =  1.99
# Ka =  4.24
# Ki =  0.0